### Notebook Goal:
The goal of this notebook is to extract a few data science and data analysis tools mentioned in the job descriptions and save it all in the MySQL database.

In [1]:
import sqlalchemy as db

import pandas as pd
import yaml
import re

### Extract tools from job description

In [2]:
# Connect to MySQL
credentials = yaml.full_load(open('./credentials.yml'))
user = credentials['database']['username']
password = credentials['database']['password']
host = credentials['database']['hostname']
database = 'indeed_jobs'

string_conexao = f'mysql://{user}:{password}@{host}/{database}'
engine = db.create_engine(string_conexao)
conn = engine.connect()
conn

In [3]:
metadata = db.MetaData()
job = db.Table('job_descriptions', metadata, autoload_with=engine)

print(job.columns.keys())

['id', 'indeed_link', 'description', 'indeed_title', 'position', 'location']


In [4]:
query = db.select(job)
results = conn.execute(query).fetchall()
df = pd.DataFrame(results)
df.head()

,id,indeed_link,description,indeed_title,position,location
0,1,https://br.indeed.com/rc/clk?jk=6ac6aac0697797...,"Somos uma empresa apaixonada por pessoas, paut...",Cientista de Dados - Junior (Remoto),Cientista de Dados,Remoto
1,2,https://br.indeed.com/rc/clk?jk=45466838932b13...,Somos uma logtech líder no segmento de transpo...,CIENTISTA DE DADOS (REMOTO),Cientista de Dados,Remoto
2,3,https://br.indeed.com/rc/clk?jk=88183160ba6b40...,"Aqui na Provu, foco, responsabilidade, coopera...",Cientista de Dados Pleno,Cientista de Dados,Remoto
3,4,https://br.indeed.com/rc/clk?jk=51a6fdcdd61c48...,O que nos move diariamente?. Todos os dias via...,PESSOA CIENTISTA DE DADOS PLENO,Cientista de Dados,Remoto
4,5,https://br.indeed.com/rc/clk?jk=41427546a9ab84...,Vaga para nosso BANCO DE TALENTOS. Procuramos ...,Cientista de Dados,Cientista de Dados,Remoto


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            449 non-null    int64 
 1   indeed_link   449 non-null    object
 2   description   449 non-null    object
 3   indeed_title  449 non-null    object
 4   position      449 non-null    object
 5   location      449 non-null    object
dtypes: int64(1), object(5)
memory usage: 21.2+ KB


In [6]:
df.iloc[1].description

'Somos uma logtech líder no segmento de transporte rodoviário de cargas para o setor. Com expertise e tecnologia de ponta, oferecemos soluções para embarcadores, frotistas e caminhoneiros que desejam digitalizar as suas operações. Responsabilidades e atribuições. Conduzir análises e experimentos sobre os dados. Construir e prototipar análises que nos permitam ter insights de otimização de negócios e até mesmo novos modelos ainda não pensados. Deverá desenvolver um conhecimento profundo do contexto da TMOV defendendo seu ponto de vista e sugerindo melhorias nos produtos e processos operacionais. Fazer recomendações de negócios com apresentações executivas e técnicas sempre priorizando formatos visuais com informações quantitativas. Pesquisar e desenvolver modelos preditivos diversos, desde recomendações de fretes, sistemas de pricing, otimizações operacionais e comerciais. Interagir com as áreas incentivando a tomada de decisões sempre baseada em dados em seus diversos níveis de alçada.

In [7]:
df.iloc[1].indeed_link

'https://br.indeed.com/rc/clk?jk=45466838932b1366&fccid=12ab49f1694345a0&vjs=3'

## Get tools with keywords only

In [8]:
tools = db.Table('tools', metadata, autoload_with=engine)

print(tools.columns.keys())

['id', 'tool', 'tool_group']


In [9]:
query = db.select(tools)
results = conn.execute(query).fetchall()
df_tools = pd.DataFrame(results)
df_tools.head()

,id,tool,tool_group
0,1,python,Linguagem
1,2,r,Linguagem
2,3,sql,Linguagem
3,4,java,Linguagem
4,5,scala,Linguagem


In [10]:
hard_skills = {'job_id': [], 'tool_id': []}

In [11]:
def remove_ponctuation_lower(x):
    x = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', x)
    return x.lower()

In [12]:
df['clean_description'] = df['description'].apply(lambda x: remove_ponctuation_lower(x))
df.iloc[1].clean_description

'somos uma logtech líder no segmento de transporte rodoviário de cargas para o setor com expertise e tecnologia de ponta oferecemos soluções para embarcadores frotistas e caminhoneiros que desejam digitalizar as suas operações responsabilidades e atribuições conduzir análises e experimentos sobre os dados construir e prototipar análises que nos permitam ter insights de otimização de negócios e até mesmo novos modelos ainda não pensados deverá desenvolver um conhecimento profundo do contexto da tmov defendendo seu ponto de vista e sugerindo melhorias nos produtos e processos operacionais fazer recomendações de negócios com apresentações executivas e técnicas sempre priorizando formatos visuais com informações quantitativas pesquisar e desenvolver modelos preditivos diversos desde recomendações de fretes sistemas de pricing otimizações operacionais e comerciais interagir com as áreas incentivando a tomada de decisões sempre baseada em dados em seus diversos níveis de alçada requisitos e 

In [13]:
for i, row in df.iterrows():
    for i, tool in df_tools.iterrows():
        if " " + tool['tool'] + " " in row['clean_description']:
            hard_skills['job_id'].append(row["id"])
            hard_skills['tool_id'].append(tool["id"])
        # I noticed a few descriptions with gcp written as google cloud, so I added this exception here.
        if tool['tool'] == "gcp":
            if " google cloud " in row['clean_description']:
                hard_skills['job_id'].append(row["id"])
                hard_skills['tool_id'].append(tool["id"])
                
df_requirements = pd.DataFrame(hard_skills).drop_duplicates().reset_index(drop=True)
df_requirements.head()

,job_id,tool_id
0,1,1
1,1,3
2,1,40
3,1,47
4,1,49


In [14]:
df_requirements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359 entries, 0 to 1358
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   job_id   1359 non-null   int64
 1   tool_id  1359 non-null   int64
dtypes: int64(2)
memory usage: 21.4 KB


### Manually random result verification

In [15]:
df.loc[df['id'] == 2].description.values[0]

'Somos uma logtech líder no segmento de transporte rodoviário de cargas para o setor. Com expertise e tecnologia de ponta, oferecemos soluções para embarcadores, frotistas e caminhoneiros que desejam digitalizar as suas operações. Responsabilidades e atribuições. Conduzir análises e experimentos sobre os dados. Construir e prototipar análises que nos permitam ter insights de otimização de negócios e até mesmo novos modelos ainda não pensados. Deverá desenvolver um conhecimento profundo do contexto da TMOV defendendo seu ponto de vista e sugerindo melhorias nos produtos e processos operacionais. Fazer recomendações de negócios com apresentações executivas e técnicas sempre priorizando formatos visuais com informações quantitativas. Pesquisar e desenvolver modelos preditivos diversos, desde recomendações de fretes, sistemas de pricing, otimizações operacionais e comerciais. Interagir com as áreas incentivando a tomada de decisões sempre baseada em dados em seus diversos níveis de alçada.

In [16]:
df_requirements.loc[df_requirements['job_id'] == 2]

,job_id,tool_id
7,2,1
8,2,3


In [17]:
df_tools.head()

,id,tool,tool_group
0,1,python,Linguagem
1,2,r,Linguagem
2,3,sql,Linguagem
3,4,java,Linguagem
4,5,scala,Linguagem


### Insert requirements into MySQL

In [18]:
metadata = db.MetaData()
requirements = db.Table('job_requirements', metadata, autoload_with=engine)

print(requirements.columns.keys())

['id', 'job_id', 'tool_id']


In [19]:
query = db.select(requirements)
results = conn.execute(query).fetchall()
current_req = pd.DataFrame(results)
current_req.drop("id", axis=1, inplace=True)
current_req.head()

,job_id,tool_id
0,1,1
1,1,3
2,1,40
3,1,47
4,1,49


In [20]:
# Remove the rows from the dataframe that are already in the DB
new_req = pd.concat([current_req, df_requirements]).drop_duplicates(keep=False)
new_req.shape

(169, 2)

In [21]:
for i, row in new_req.iterrows():
    try:
        stmt = db.insert(requirements).values(job_id=row['job_id'], tool_id=row['tool_id'])

        conn.execute(stmt) 
        conn.commit()    
    except Exception as e:        
        print(f"Não foi possível inserir o regristro {i}. O erro foi encontrado foi: {e}")